Copyright 2019 Google LLC

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

In [ ]:
!git clone https://github.com/google-research/google-research.git

In [1]:
import sys
import os
import tarfile
import urllib
sys.path.append('./google-research')

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # no need to use gpu

In [3]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1
import logging
from kws_streaming.models import models
from kws_streaming.train import model_flags
from kws_streaming.layers.modes import Modes
from kws_streaming.train import test
from kws_streaming.models import utils
from kws_streaming.data import input_data
from kws_streaming.models import model_params

In [4]:
tf1.__version__

'2.1.0'

In [5]:
config = tf1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf1.Session(config=config)

In [6]:
tf1.disable_eager_execution()

In [7]:
DATA_VERSION = 1

In [8]:
if DATA_VERSION == 2:
  DATA_URL = "https://storage.googleapis.com/download.tensorflow.org/data/speech_commands_v0.02.tar.gz"
  DATA_PATH = "/home/tmp/kws_streaming/data22/"
elif DATA_VERSION == 1:
  DATA_URL = "http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz"
  DATA_PATH = "/home/tmp/kws_streaming/data11/"
else:
  assert(False)

In [9]:
base_name = os.path.basename(DATA_URL)
base_name

'speech_commands_v0.01.tar.gz'

In [10]:
arch_file_name = os.path.join(DATA_PATH, base_name)
if not os.path.isfile(arch_file_name):
  # download data
  if sys.version_info >= (2, 5):
    file_path = urllib.request.urlretrieve(DATA_URL, filename=arch_file_name)[0]
  else:
    file_path = urllib.urlretrieve(DATA_URL, filename=arch_file_name)[0]

  # unpack it
  file_name, file_extension = os.path.splitext(base_name)
  tar = tarfile.open(file_path)
  tar.extractall(DATA_PATH)  
  tar.close()

In [13]:
# default parameters for data splitting
flags = model_params.Params()
flags.data_dir = DATA_PATH
flags.data_url = DATA_URL
flags = model_flags.update_flags(flags)

In [14]:
audio_processor = input_data.AudioProcessor(flags)

In [15]:
testing_set_size = audio_processor.set_size('testing')
print("testing_set_size " + str(testing_set_size))
training_set_size = audio_processor.set_size('training')
print("training_set_size " + str(training_set_size))
validation_set_size = audio_processor.set_size('validation')
print("validation_set_size " + str(validation_set_size))

testing_set_size 3081
training_set_size 22246
validation_set_size 3093


In [16]:
# V2
# testing_set_size 4890
# training_set_size 36923
# validation_set_size 4445

# V1
# testing_set_size 3081
# training_set_size 22246
# validation_set_size 3093

In [17]:
# all words used for modeling: we have target words + unknown words (with index 1)
audio_processor.word_to_index

{'down': 5,
 'four': 1,
 'sheila': 1,
 'three': 1,
 'one': 1,
 'go': 11,
 'no': 3,
 'right': 7,
 'wow': 1,
 'six': 1,
 'happy': 1,
 'on': 8,
 'marvin': 1,
 'five': 1,
 'left': 6,
 'seven': 1,
 'bird': 1,
 'yes': 2,
 'off': 9,
 'zero': 1,
 'eight': 1,
 'up': 4,
 'bed': 1,
 'cat': 1,
 'stop': 10,
 'two': 1,
 'dog': 1,
 'house': 1,
 'nine': 1,
 'tree': 1,
 '_silence_': 0}

In [19]:
# find the start of the file name where label begins
string = audio_processor.data_index["validation"][0]['file']
res = [i for i in range(len(string)) if string.startswith('/', i)] 
start_file = res[-2]+1
start_file

32

In [21]:
audio_processor.data_index["validation"][0]['file'][start_file:]

'down/d197e3ae_nohash_0.wav'

In [23]:
index_to_label = {}
unknown_category = []
# labeles used for training
for word in audio_processor.word_to_index.keys():
  if audio_processor.word_to_index[word] == input_data.SILENCE_INDEX:
    index_to_label[audio_processor.word_to_index[word]] = input_data.SILENCE_LABEL
  elif audio_processor.word_to_index[word] == input_data.UNKNOWN_WORD_INDEX:
    index_to_label[audio_processor.word_to_index[word]] = input_data.UNKNOWN_WORD_LABEL
    unknown_category.append(word)
  else:
    index_to_label[audio_processor.word_to_index[word]] = word

# training labels
index_to_label

{5: 'down',
 1: '_unknown_',
 11: 'go',
 3: 'no',
 7: 'right',
 8: 'on',
 6: 'left',
 2: 'yes',
 9: 'off',
 4: 'up',
 10: 'stop',
 0: '_silence_'}

In [24]:
# words belonging to unknown categry
unknown_category

['four',
 'sheila',
 'three',
 'one',
 'wow',
 'six',
 'happy',
 'marvin',
 'five',
 'seven',
 'bird',
 'zero',
 'eight',
 'bed',
 'cat',
 'two',
 'dog',
 'house',
 'nine',
 'tree']

In [26]:
def get_distribution(mode):
  distrib_label = {}
  distrib_words = {}
  files = {}
  for data in audio_processor.data_index[mode]:
    word = data['label']
    file = data['file'][start_file:]
    index = audio_processor.word_to_index[word]
    label = index_to_label[index]
    if word in files:
      files[word].append(file)
    else:
      files[word] = [file]
    
    if label in distrib_label:
      distrib_label[label] = distrib_label[label] + 1
    else:
      distrib_label[label] = 0
      
    if word in distrib_words:
      distrib_words[word] = distrib_words[word] + 1
    else:
      distrib_words[word] = 0      
  return distrib_words, distrib_label, files

In [27]:
# distribution of labeles in testing data
distrib_words_testing, distrib_labels_testing, files_testing = get_distribution('testing')
distrib_labels_testing

{'_unknown_': 256,
 'go': 250,
 'yes': 255,
 'on': 245,
 'off': 261,
 'down': 252,
 'no': 251,
 'stop': 248,
 'up': 271,
 'right': 258,
 'left': 266,
 '_silence_': 256}

In [32]:
# distribution of labeles in training data
distrib_words_training, distrib_labels_training, files_training = get_distribution('training')
distrib_labels_training

{'_unknown_': 1853,
 'on': 1863,
 'stop': 1884,
 '_silence_': 1853,
 'no': 1852,
 'go': 1860,
 'up': 1842,
 'left': 1838,
 'down': 1841,
 'off': 1838,
 'yes': 1859,
 'right': 1851}

In [39]:
def parse_files(set_list_fname, label='yes'):
  set_files = []
  with open(set_list_fname) as f:
    while True:
      line = f.readline()
      if not line:
        break
      if line.split('/')[0]==label:
        set_files.append(line[:-1])    
  return set_files

In [48]:
def validate(my_list1, list2, print_in_list2=False):
  cnt_my_val2 = 0
  cnt_my_val1 = 0
  for my_val in my_list1:
    if my_val in list2:
      cnt_my_val2 = cnt_my_val2 + 1
      if print_in_list2:
        print(my_val)
    else:
      cnt_my_val1 = cnt_my_val1 + 1
      if not print_in_list2:
        print(my_val)
  return cnt_my_val1, cnt_my_val2

In [49]:
file_list = os.path.join(DATA_PATH, "testing_list.txt")

# validate that all wav used during testing belongs to testing_list
for word in files_testing.keys():
  if word != '_silence_':
    word_files = parse_files(file_list, label=word)
    _, cnt_val = validate(files_testing[word], word_files, False)
    assert(cnt_val-len(files_testing[word])==0)

In [50]:

distrib_words_training, distrib_labels_training, files_training = get_distribution('training')

# validate that all wav used during testing do not belong to training data
for word in files_testing.keys():
  if word != '_silence_': # silence file does not matter becasue it is multiplied by zero
    word_files = files_testing[word]
    _, cnt_val = validate(files_training[word], word_files, True)
    assert(cnt_val==0)